<a href="https://colab.research.google.com/github/NanditaJaiswal12/nanditajaiswal12/blob/main/Spotify_Music_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 6.2 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import requests
import base64
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
# Create spotify account if you don't have the account (You don't need to purchase any subscription to get your credentials).
# Log in to your account from your web browser.
# Next you need to log in to your Spotify developer dashboard (you will be using this developer account for the first time, sign the agreement and verify your email).
# Once the email is verified, you will see an option to create an app in your dashboard.
# Click “Create app” and fill the app description.
# After filling in the app description, you will be redirected to your app dashboard.
# Here, you will see your Client ID.
# Click on the "Show Client Secret" button to view your Client Secret.
# Make sure to copy these values and keep them in a secure place as you will need them for authentication in your application.

# Add your Client ID and Client Secret
CLIENT_ID = 'd148c14780b84495b768318410b91be5'
CLIENT_SECRET = '6910fdf1fd0a4f41afdf89b23241b8aa'

In [ ]:
# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode()).decode()

In [ ]:
# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {'Authorization': f'Basic {client_credentials_base64}'}
data = {'grant_type': 'client_credentials'}

# Send POST request to Spotify API to get the access token
response = requests.post(token_url, headers=headers, data=data)
access_token = response.json().get('access_token')

if access_token:
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()


Access token obtained successfully.


In [ ]:
# Function to get playlist data from Spotify
def get_playlist_data(playlist_id, access_token):
    # Initialize Spotify client
    sp = spotipy.Spotify(auth=access_token)

    # Get tracks from the specified playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items.track(id, name, artists, album(id, name, release_date), popularity)')

    tracks_data = []
    for item in playlist_tracks['items']:
        track = item['track']
        track_info = {
            'Track Name': track['name'],
            'Artists': ', '.join(artist['name'] for artist in track['artists']),
            'Album Name': track['album']['name'],
            'Album ID': track['album']['id'],
            'Track ID': track['id'],
            'Popularity': track['popularity'],
            'Release Date': track['album']['release_date'],
        }

        # Get audio features for the track
        audio_features = sp.audio_features(track['id'])[0]
        if audio_features:
            track_info.update({
                'Danceability': audio_features['danceability'],
                'Energy': audio_features['energy'],
                'Key': audio_features['key'],
                'Loudness': audio_features['loudness'],
                'Mode': audio_features['mode'],
                'Speechiness': audio_features['speechiness'],
                'Acousticness': audio_features['acousticness'],
                'Instrumentalness': audio_features['instrumentalness'],
                'Liveness': audio_features['liveness'],
                'Valence': audio_features['valence'],
                'Tempo': audio_features['tempo']
            })
        tracks_data.append(track_info)

    return pd.DataFrame(tracks_data)


In [ ]:
# Find the Playlist on Spotify.
# Copy the Playlist Link (Click on share option and Click on "Copy link to playlist" or "Copy Spotify URI").
# Extract the Playlist ID
# Your spotify link will look like this: 'https://open.spotify.com/playlist/37i9dQZF1DXcBWIGoYBM5M' and your spotify playlist ID will be '37i9dQZF1DXcBWIGoYBM5M'.

# Add your Playlist ID here
playlist_id = '3HUlMvsYeGujI29XTbMiAA'

In [ ]:
# Get the playlist data
music_df = get_playlist_data(playlist_id, access_token)

music_df

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,"Raataan Lambiyan (From ""Shershaah"")","Tanishk Bagchi, Jubin Nautiyal, Asees Kaur","Raataan Lambiyan (From ""Shershaah"")",2DKSZvMj7Da6rzNVPMgREj,2rOnSn2piaqLAlYjtfUBlY,70,2021-07-30,0.720,0.630,10,-5.549,1,0.0413,0.6150,0.000000,0.108,0.512,81.009
1,,,,2HSFoyWN8n9c3f2SMceRrO,0ZHILXmUaVSLEOCWxPQ1ga,0,0000,0.696,0.704,1,-7.876,1,0.0423,0.1670,0.000000,0.155,0.426,124.972
2,Raabta,"Pritam, Arijit Singh",Agent Vinod,2DqQ34i4uuuZWTScsGIgHr,6FjbAnaPRPwiP3sciEYctO,70,2012-02-24,0.657,0.581,0,-8.108,1,0.0298,0.6990,0.000018,0.116,0.323,118.037
3,"Kaise Hua (From ""Kabir Singh"")","Vishal Mishra, Manoj Muntashir","Kaise Hua (From ""Kabir Singh"")",1ncyKqyZae6DoyjYaV9LmS,1y1rQTkWmrZdJmjwuK07GC,72,2019-06-13,0.385,0.464,2,-6.975,1,0.0404,0.4520,0.000000,0.109,0.150,137.839
4,Shayad,"Pritam, Arijit Singh",Love Aaj Kal (Original Motion Picture Soundtrack),6IXfWhC7omotM4lt0ROmEr,1smFN2CLqGROu0J0UyvDfL,72,2020-02-14,0.483,0.468,8,-7.642,1,0.0341,0.7840,0.000000,0.152,0.319,135.875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Kesariya (From ""Brahmastra"")","Pritam, Arijit Singh, Amitabh Bhattacharya","Kesariya (From ""Brahmastra"")",1HeX4SmCFW4EPHQDvHgrVS,6VBhH7CyP56BXjp8VsDFPZ,72,2022-07-17,0.577,0.574,0,-6.525,1,0.0278,0.5720,0.000000,0.100,0.436,94.009
96,Lamhey,Anubha Bajaj,Lamhey,1jZWNePo4dPSElLKJ0I2zA,5Z2GQ0eBKoatNYqbGayihP,63,2023-02-24,0.571,0.395,2,-10.502,1,0.0372,0.6010,0.000247,0.255,0.631,173.998
97,"Besharam Rang (From ""Pathaan"")","Vishal-Shekhar, Shilpa Rao, Caralisa Monteiro,...","Besharam Rang (From ""Pathaan"")",5CvI5ppsG522pV554VLezD,0CtZpaOhtzvLV3FfcsVpQo,68,2022-12-12,0.773,0.795,8,-5.537,0,0.0412,0.0587,0.002370,0.154,0.649,115.997
98,Jhoome Jo Pathaan,"Vishal-Shekhar, Arijit Singh, Sukriti Kakar, V...",Pathaan,2ooWY7xriwwyj23dAI32C2,6FAYpZ4jve8vpvTwUvjK6H,67,2022-12-22,0.817,0.738,7,-7.639,1,0.0748,0.0964,0.000000,0.331,0.616,104.964


In [ ]:
# Data Preprocessing
music_df.isnull().sum()

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64

In [ ]:
# Normalize the features
scaler = MinMaxScaler()
features = ['Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo']
music_df[features] = scaler.fit_transform(music_df[features])


In [ ]:
# Content-Based Recommendation System
def content_based_recommendations(input_song_name, num_recommendations=5):
    # Check if the input song is in the dataset
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate similarity scores based on normalized features
    similarity_scores = cosine_similarity([music_df.loc[input_song_index, features]], music_df[features])[0]
    # Get the indices of the most similar songs
    similar_songs_indices = similarity_scores.argsort()[::-1][1:num_recommendations+1]

    # Return the most similar songs
    return music_df.iloc[similar_songs_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]


In [ ]:
# Hybrid Recommendation System
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')
    # Calculate the time span between the release date and today
    time_span = (datetime.now() - release_date).days
    # Calculate the weight based on the time span ((more recent releases have higher weight))
    weight = 1 / (time_span + 1)
    return weight

def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    # Check if the input song is in the dataset
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_rec = content_based_recommendations(input_song_name, num_recommendations)
    # Get the index of the input song
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]
    # Get the popularity score of the iput song
    popularity_score = music_df.loc[input_song_index, 'Popularity']
    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[input_song_index, 'Release Date'])

    # Combine the content-based recommendations with the weighted popularity score
    hybrid_recs = content_rec.copy()
    hybrid_recs.loc[len(hybrid_recs)] = music_df.loc[input_song_index, ['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']].to_dict()
    hybrid_recs['Popularity'] = hybrid_recs.apply(lambda row: row['Popularity'] * calculate_weighted_popularity(row['Release Date']), axis=1)
    hybrid_recs = hybrid_recs.sort_values(by='Popularity', ascending=False)
    # Remove the input song from the recommendations
    hybrid_recs = hybrid_recs[hybrid_recs['Track Name'] != input_song_name]

    return hybrid_recs


In [ ]:
# Example usage
input_song_name = 'Kesariya (From "Brahmastra")'
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)

# Print the recommended songs
print(f"Hybrid recommended songs for '{input_song_name}':")
recommendations

Hybrid recommended songs for 'Kesariya (From "Brahmastra")':


,Track Name,Artists,Album Name,Release Date,Popularity
61,Hawayein,"Pritam, Arijit Singh",Jab Harry Met Sejal (Original Motion Picture S...,2017-08-03,0.027975
72,Kinna Sona,Sunil Kamath,Bhaag Johnny,2015-08-24,0.018473
2,Raabta,"Pritam, Arijit Singh",Agent Vinod,2012-02-24,0.015470
54,Mere Bina,"Pritam, Nikhil D'Souza",Crook (Original Motion Picture Soundtrack),2010-09-03,0.013033
